In [2]:
import pandas as pd
from google.colab import files

# Upload the file manually in Google Colab
uploaded = files.upload()
file_name = list(uploaded.keys())[0]  # Get uploaded file name

# Load the Excel file
xls = pd.ExcelFile(file_name)

# Load the sheet, skipping unnecessary rows
df = pd.read_excel(xls, sheet_name='Sheet1', skiprows=2)

# Print the actual column names
print("Actual column names:", df.columns)

# Rename relevant columns
column_mapping = {
    df.columns[2].strip(): 'Source Head',
    df.columns[3].strip(): 'Final Source',
    df.columns[4].strip(): 'Qualified Leads',
    df.columns[5].strip(): 'Week 2 Qualified Leads',
    df.columns[6].strip(): 'Week 3 Qualified Leads',
    df.columns[8].strip(): 'Total Conversions',
    df.columns[9].strip(): 'Week 2 Total Conversions',
    df.columns[10].strip(): 'Week 3 Total Conversions',
    df.columns[12].strip(): 'Cost',
    df.columns[13].strip(): 'Week 2 Cost',
    df.columns[14].strip(): 'Week 3 Cost'
}
df.rename(columns=column_mapping, inplace=True)

# Drop irrelevant columns
columns_to_drop = [df.columns[0], df.columns[1]]
df.drop(columns=columns_to_drop, inplace=True)

# Forward fill 'Source Head' to fill merged cells
df['Source Head'].fillna(method='ffill', inplace=True)

# Remove any empty rows
df = df[df['Final Source'].notna()]

# Reshape the dataset into the desired format
reshaped_data = []
weeks = ['Week 1', 'Week 2', 'Week 3']
for index, row in df.iterrows():
    source_head = row['Source Head']
    final_source = row['Final Source']
    for week_idx, week in enumerate(weeks):
        reshaped_data.append({
            'Source Head': source_head,
            'Final Source': final_source,
            'Week': week,
            'Qualified Leads': row[f'Qualified Leads'] if week_idx == 0 else row[f'Week {week_idx+1} Qualified Leads'],
            'Total Conversions': row[f'Total Conversions'] if week_idx == 0 else row[f'Week {week_idx+1} Total Conversions'],
            'Cost': row[f'Cost'] if week_idx == 0 else row[f'Week {week_idx+1} Cost']
        })

# Convert to DataFrame
df_final = pd.DataFrame(reshaped_data)

# Ensure proper data types
df_final['Qualified Leads'] = pd.to_numeric(df_final['Qualified Leads'], errors='coerce')
df_final['Total Conversions'] = pd.to_numeric(df_final['Total Conversions'], errors='coerce')
df_final['Cost'] = pd.to_numeric(df_final['Cost'], errors='coerce')

# Remove unwanted values
unwanted_source_heads = ['Reference']
unwanted_final_sources = ['TV Total', 'Digital Total']
df_final = df_final[~df_final['Source Head'].isin(unwanted_source_heads)]
df_final = df_final[~df_final['Final Source'].isin(unwanted_final_sources)]

# Ensure proper week ordering
df_final['Week'] = pd.Categorical(df_final['Week'], categories=weeks, ordered=True)
df_final.sort_values(by=['Source Head', 'Final Source', 'Week'], inplace=True)

# Save the cleaned data to an Excel file
output_filename = "Cleaned_Data.xlsx"
df_final.to_excel(output_filename, index=False)

# Download the file
files.download(output_filename)

Saving Raw Data.xlsx to Raw Data (2).xlsx
Actual column names: Index(['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3',
       'Qualified Leads', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7',
       'Total landed conversions', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11',
       'Cost', 'Unnamed: 13', 'Unnamed: 14', 'COA', 'Unnamed: 16',
       'Unnamed: 17'],
      dtype='object')


<ipython-input-2-42f78ffa98a4>:38: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Source Head'].fillna(method='ffill', inplace=True)
<ipython-input-2-42f78ffa98a4>:38: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['Source Head'].fillna(method='ffill', inplace=True)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>